In [1]:
# simple convnet
from keras import layers
from keras import models

model = models.Sequential()
# (image_height, image_width, image_channels) 크기의 3D 입력텐서
# MNIST 이미지 포캣 (28, 28, 1) 크기의 입력 처리하도록
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape= (28, 28, 1)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()
# height, width 차원은 네트워크 깊어질수록 작아짐(26->13->11->5->3)

Metal device set to: Apple M1


2022-03-20 15:16:52.448807: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-20 15:16:52.449047: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-traina

In [2]:
# convnet 위에 분류기 추가
model.add(layers.Flatten()) # 이전 출력층 3D 텐서를 1D 텐서로 펼치기
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax')) # 10개의 클래스 구분

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

In [4]:
# mnist 이미지에 convnet 훈련
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5


2022-03-20 15:37:14.788762: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-20 15:37:15.077241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


938/938 [==============================] - 15s 12ms/step - loss: 0.1791 - acc: 0.9436
Epoch 2/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0466 - acc: 0.9855
Epoch 3/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0325 - acc: 0.9901
Epoch 4/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0245 - acc: 0.9925
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0188 - acc: 0.9941


In [5]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

 30/313 [=>............................] - ETA: 1s - loss: 0.0391 - acc: 0.9833

2022-03-20 15:38:53.761325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 6ms/step - loss: 0.0270 - acc: 0.9922


0.9922000765800476